In [70]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [71]:
df = pd.read_csv('KaggleCovidDataset.csv').head().applymap(lambda x : 1 if x == "Yes" else 0)
df

/tmp/ipykernel_28619/3230690699.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = pd.read_csv('KaggleCovidDataset.csv').head().applymap(lambda x : 1 if x == "Yes" else 0)


,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,1,1,1,1,1,0,0,0,0,1,...,1,1,0,1,0,1,1,0,0,1
1,1,1,1,1,0,1,1,1,0,0,...,1,0,0,0,1,1,0,0,0,1
2,1,1,1,1,1,1,1,1,0,1,...,1,1,1,0,0,0,0,0,0,1
3,1,1,1,0,0,1,0,0,1,1,...,0,0,1,0,1,1,0,0,0,1
4,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,1,0,0,0,1


In [72]:
def aprioriBAD(df, min_support):
    # Generate 1-itemsets
    oneItemSets = df.apply(lambda x: x.value_counts(normalize=True).get(1, 0)).loc[lambda x: x > min_support].to_dict()

    # Function to generate k-itemsets
    def generate_k_itemset(itemset, k):
        itemset_k = []
        for item1 in itemset:
            for item2 in itemset:
                if item1[:-1] == item2[:-1] and item1[-1] != item2[-1]:
                    new_item = tuple(sorted(set(item1 + item2)))
                    if len(new_item) == k:
                        itemset_k.append(new_item)
        itemset_k = list(set(itemset_k))
        itemset_k_support_map = {}
        for kitem in itemset_k:
            countPresent = 0
            for doc in range(df.shape[0]):
                addCount = True
                for item in kitem:
                    if df[item][doc] == 0:
                        addCount = False
                if addCount:
                    countPresent += 1
            support = countPresent/df.shape[0]
            if support > min_support:
                itemset_k_support_map[frozenset(kitem)] = support
        return itemset_k_support_map

    # Generate 2-itemsets
    twoItemSets = generate_k_itemset([tuple([key]) for key in oneItemSets.keys()], 2)

    # Generate k-itemsets for k > 2
    kItemSets = twoItemSets
    k = 3
    while len(kItemSets) > 0:
        kItemSets = generate_k_itemset([tuple(key) for key in kItemSets.keys()], k)
        k += 1

    # Combine all itemsets
    allItemSets = {**oneItemSets, **twoItemSets, **kItemSets}

    # Convert to DataFrame
    result = pd.DataFrame([(v, k) for k, v in allItemSets.items()], columns=['support', 'itemsets'])
    return result


In [73]:
aprioriBAD(df, min_support=0.6)

,support,itemsets
0,1.0,Breathing Problem
1,1.0,Fever
2,1.0,Dry Cough
3,0.8,Sore throat
4,0.8,Diabetes
5,0.8,Visited Public Exposed Places
6,1.0,COVID-19
7,1.0,"(Breathing Problem, Fever)"
8,0.8,"(Visited Public Exposed Places, Fever)"
9,0.8,"(Visited Public Exposed Places, Breathing Prob..."
